In [ ]:
import requests
import json
from datetime import datetime, timezone  # <-- aggiungi

CAPTURE_URL = "http://localhost:8080/epcis/capture"  # <-- usa il mock

def send_event(event):
    response = requests.post(CAPTURE_URL, json=event)
    biz_step = event["epcisBody"]["eventList"][0]["bizStep"]
    print(f"Invio evento '{biz_step}'... Status: {response.status_code}")

def create_object_event(epc, biz_step, disposition, read_point, biz_location, extra={}):
    return {
        "type": "EPCISDocument",
        "schemaVersion": "2.0",
        "creationDate": datetime.now(timezone.utc).isoformat(timespec="seconds"),
        "epcisBody": {
            "eventList": [
                {
                    "type": "ObjectEvent",
                    "action": "ADD",
                    "eventTime": datetime.now(timezone.utc).isoformat(timespec="seconds") + "Z",
                    "eventTimeZoneOffset": "+02:00",
                    "epcList": [epc],
                    "bizStep": biz_step,
                    "disposition": disposition,
                    "readPoint": { "id": read_point },
                    "bizLocation": { "id": biz_location },
                    "ilmd": extra
                }
            ]
        }
    }

def make_certificate(scheme, code, holder, scope, valid_from, valid_to, certifier, standard=None, statement=None, license_url=None):
    cert = {
        "scheme": scheme,                  # es: "FSC" | "PEFC"
        "code": code,                      # es: "FSC C168356"
        "holder": holder,                  # es: "Azienda XYZ s.r.l."
        "scope": scope,                    # es: "Chain of Custody"
        "validFrom": valid_from,           # "YYYY-MM-DD"
        "validTo": valid_to,               # "YYYY-MM-DD"
        "certifier": certifier             # organismo accreditato
    }
    if standard: cert["standard"] = standard
    if statement: cert["statement"] = statement
    if license_url: cert["licenseUrl"] = license_url
    return cert

# --- CORREZIONE: Codice spostato fuori dall'if __name__ == "__main__" ---

# 1. Taglio albero
print("--- Invio eventi al server ---")
e1 = create_object_event(
    epc="urn:epc:id:lot:wood.LOT20250901",
    biz_step="harvesting",
    disposition="active",
    read_point="urn:epc:id:sgln:1234567890001.0.0",
    biz_location="urn:epc:id:sgln:1234567890001.0.0",
    extra={"palm:certification": "PEFC-IT/21-31-567"}
)
send_event(e1)

# 1.1. Certificazione legno (arricchita con dettagli FSC)
e_cert = create_object_event(
    epc="urn:epc:id:lot:wood.LOT20250901",
    biz_step="certification",
    disposition="active",
    read_point="urn:epc:id:sgln:1234567890001.0.0",
    biz_location="urn:epc:id:sgln:1234567890001.0.0",
    extra={
        # elenco di certificazioni strutturate
        "palm:certificates": [
            make_certificate(
                scheme="FSC",
                code="FSC C168356",
                holder="Azienda XYZ s.r.l.",
                scope="Chain of Custody",
                valid_from="2024-06-15",
                valid_to="2029-06-14",
                certifier="Nome organismo accreditato",
                standard="FSC-STD-40-004",
                statement="Products originate from forests managed in accordance with FSC Principles and Criteria."
            )
            # puoi aggiungere anche PEFC qui, come seconda voce della lista
            # make_certificate("PEFC", "PEFC-IT/21-31-567", "Azienda XYZ s.r.l.", "Chain of Custody", "2024-06-15", "2029-06-14", "Org. accreditato")
        ],
        # versione “testo” pronta da mostrare
        "palm:certificateText_it": (
            "FOREST STEWARDSHIP COUNCIL®\n"
            "Certificate Code: FSC C168356\n"
            "Certificate Holder: Azienda XYZ s.r.l.\n"
            "Scope: Chain of Custody\n"
            "Valid from: 15/06/2024   Valid to: 14/06/2029\n"
            "Certifier: Nome organismo accreditato\n"
            "This certifies that the products originate from forests managed "
            "in accordance with FSC Principles and Criteria, COC Standard, etc."
        )
    }
)
send_event(e_cert)

# 2. Trasporto in segheria
e2 = create_object_event(
    epc="urn:epc:id:lot:wood.LOT20250901",
    biz_step="transporting",
    disposition="in_transit",
    read_point="urn:epc:id:sgln:1234567890002.0.0",
    biz_location="urn:epc:id:sgln:1234567890003.0.0"
)
send_event(e2)

# 3. Assemblaggio pallet in Palm
e3 = create_object_event(
    epc="urn:epc:id:sscc:1234567.0000000001",
    biz_step="assembling",
    disposition="active",
    read_point="urn:epc:id:sgln:1234567890005.0.0",
    biz_location="urn:epc:id:sgln:1234567890005.0.0",
    extra={"palm:woodLot": "LOT20250901", "palm:modelType": "EUR-EPAL"}
)
send_event(e3)

# 4. Spedizione pallet al cliente
e4 = create_object_event(
    epc="urn:epc:id:sscc:1234567.0000000001",
    biz_step="shipping",
    disposition="in_transit",
    read_point="urn:epc:id:sgln:1234567890005.0.0",
    biz_location="urn:epc:id:sgln:1234567891116.0.0"
)
send_event(e4)
print("--- Tutti gli eventi sono stati inviati. ---")

In [ ]:
import requests
import json
from datetime import datetime, timezone  # <-- aggiungi

def _fmt_time(ts):
    if not ts:
        return "n/d"
    ts = ts.replace("Z", "+00:00")
    try:
        dt = datetime.fromisoformat(ts).astimezone()
        return dt.strftime("%d/%m/%Y %H:%M:%S")
    except Exception:
        return ts

def _humanize_step(step):
    if not step:
        return "n/d"
    mapping = {
        "harvesting": "Taglio",
        "transporting": "Trasporto",
        "assembling": "Assemblaggio",
        "shipping": "Spedizione",
        "receiving": "Ricezione",
        "certification": "Certificazione",
        #qui aggiungi la traduzione degli eventi,
    }
    # se è una URI CBV, prendi l'ultimo segmento
    label = step.split(":")[-1]
    return mapping.get(step, mapping.get(label, label))

def _humanize_disposition(disp):
    if not disp:
        return "n/d"
    mapping = {
        "active": "Attivo",
        "in_transit": "In transito",
        "damaged": "Danneggiato",
        "expired": "Scaduto",
    }
    label = disp.split(":")[-1]
    return mapping.get(disp, mapping.get(label, label))

def _short(uri):
    if not uri:
        return "n/d"
    # prendi l'ultimo pezzo utile
    for sep in ("/", ":", "."):
        if sep in uri:
            uri = uri.split(sep)[-1]
    return uri

def _humanize_ilmd_key(key):
    mapping = {
        "palm:woodLot": "Lotto legno",
        "palm:modelType": "Modello pallet",
        "palm:originCountry": "Paese di origine",
        "palm:woodSpecies": "Specie legnosa",
        "palm:receivedBy": "Ricevuto da",
        "palm:notes": "Note",
        "palm:certification": "Certificazione",
        "palm:certificateText_it": "Certificato (IT)",
    }
    return mapping.get(key, key)

def _print_ilmd_human(ilmd):
    # Gestione stringhe che contengono JSON o JSON concatenati "}{"
    if isinstance(ilmd, str):
        s = ilmd.strip()
        try:
            parsed = json.loads(s)
            return _print_ilmd_human(parsed)
        except Exception:
            try:
                parsed = json.loads(f'[{s.replace("}{", "},{")}]')
                for item in parsed:
                    _print_ilmd_human(item)
                return
            except Exception:
                print("Testo ILMD:")
                for line in s.splitlines():
                    print("  " + line)
                return

    # Dizionario: stampa campi noti in modo leggibile
    if isinstance(ilmd, dict):
        handled = set()

        if "palm:certificateText_it" in ilmd:
            print("Certificato (IT):")
            for line in str(ilmd["palm:certificateText_it"]).splitlines():
                print("  " + line)
            handled.add("palm:certificateText_it")

        if "palm:certification" in ilmd:
            print(f"Certificazione: {ilmd['palm:certification']}")
            handled.add("palm:certification")

        if isinstance(ilmd.get("palm:certificates"), list):
            print("Certificazioni:")
            for c in ilmd["palm:certificates"]:
                if isinstance(c, dict):
                    scheme = c.get("scheme", "")
                    code = c.get("code", "")
                    vf, vt = c.get("validFrom"), c.get("validTo")
                    certifier = c.get("certifier")
                    parts = [f"- {scheme} {code}".strip()]
                    if vf or vt:
                        parts.append(f"(validità {vf or '?'}–{vt or '?'})")
                    if certifier:
                        parts.append(f"Certificatore: {certifier}")
                    print("  " + "  ".join(parts))
                    if c.get("standard"):
                        print(f"    Standard: {c['standard']}")
                    if c.get("statement"):
                        print(f"    Dichiarazione: {c['statement']}")
                else:
                    print(f"  - {c}")
            handled.add("palm:certificates")

        # Altri campi generici
        for k, v in ilmd.items():
            if k in handled:
                continue
            print(f"{_humanize_ilmd_key(k)}: {v}")
        return

    # Lista: stampa ogni elemento
    if isinstance(ilmd, list):
        for item in ilmd:
            _print_ilmd_human(item)
        return

def _print_event_card(event, idx):
    when = _fmt_time(event.get("eventTime"))
    step = _humanize_step(event.get("bizStep"))
    disp = _humanize_disposition(event.get("disposition"))
    rp = _short((event.get("readPoint") or {}).get("id"))
    bl = _short((event.get("bizLocation") or {}).get("id"))
    ilmd = event.get("ilmd") or {}

    line = "-" * 60
    print(f"\n{line}\nEvento {idx}")
    print(f"Data e ora:     {when}")
    print(f"Fase:           {step}")
    print(f"Stato:          {disp}")
    print(f"Punto lettura:  {rp}")
    print(f"Luogo:          {bl}")
    if ilmd:
        print("Dettagli prodotto/lotto (ILMD):")
        _print_ilmd_human(ilmd)   # <--- stampa human readable
    print(line)

def _extract_wood_lot_from_events(wrappers):
    # Cerca palm:woodLot negli ILMD degli eventi (partendo dai più recenti)
    for w in reversed(wrappers):
        ev = w.get("epcisBody", {}).get("eventList", [{}])[0]
        ilmd = ev.get("ilmd") or {}
        lot = ilmd.get("palm:woodLot") or ilmd.get("woodLot")
        if lot:
            return lot
    return None

def _to_lot_epc(val):
    # Se non è già un URN completo, prefissa lo schema del tuo esempio
    return val if str(val).startswith("urn:") else f"urn:epc:id:lot:wood.{val}"

def query_events(epc, include_related_lot=True):
    QUERY_URL = "http://localhost:8080/epcis/events"
    headers = {"Accept": "application/ld+json"}

    print(f"🔍 Sto interrogando il server per l'EPC: {epc}...")
    try:
        r = requests.get(QUERY_URL, params={"EQ_epc": epc}, headers=headers, timeout=5)
        r.raise_for_status()
        data = r.json() if r.text else {}
        wrappers = data.get("member", [])

        # Se richiesto, recupera anche gli eventi del lotto collegato (palm:woodLot)
        if include_related_lot:
            lot_val = _extract_wood_lot_from_events(wrappers)
            if lot_val:
                lot_epc = _to_lot_epc(lot_val)
                if lot_epc != epc:
                    print(f"➕ Trovato lotto collegato: {lot_epc}. Aggiungo anche i suoi eventi...")
                    r2 = requests.get(QUERY_URL, params={"EQ_epc": lot_epc}, headers=headers, timeout=5)
                    if r2.ok and r2.text:
                        wrappers += (r2.json().get("member", []))

        if not wrappers:
            print(f"✅ Nessun evento trovato.")
            return None

        # Appiattisci e ordina per data
        flat = []
        for w in wrappers:
            ev = w.get("epcisBody", {}).get("eventList", [{}])[0]
            if ev:
                flat.append(ev)

        flat.sort(key=lambda e: e.get("eventTime", ""))  # crescente per data

        print(f"\n📦 Trovati {len(flat)} eventi (inclusi eventuali eventi del lotto):")
        for i, ev in enumerate(flat, start=1):
            _print_event_card(ev, i)

        last = flat[-1]
        print("\n🧭 Riepilogo:")
        print(f"- Ultimo aggiornamento: {_fmt_time(last.get('eventTime'))}")
        print(f"- Ultima fase:          {_humanize_step(last.get('bizStep'))}")
        print(f"- Stato attuale:        {_humanize_disposition(last.get('disposition'))}")

        return {"type": "Collection", "member": [{"epcisBody": {"eventList": [e]}} for e in flat]}

    except requests.exceptions.JSONDecodeError:
        print("--- ERRORE DI DECODIFICA JSON ---")
        print(r.text)
        return None
    except requests.exceptions.RequestException as e:
        print("--- ERRORE DI CONNESSIONE ---")
        print(f"Dettaglio: {e}")
        return None

# Chiamata alla funzione
pallet_epc = "urn:epc:id:sscc:1234567.0000000001"
query_events(pallet_epc)